<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_APIChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain + OpenAI实现基于自然语言的网络爬虫

In [1]:
# !pip install -qU langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 696.4/696.4 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 509.3 kB/s eta 0:00:00


In [1]:
import os,sys
import openai
from dotenv import load_dotenv, find_dotenv
# sys.path.append("../..")

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
print(find_dotenv())
_ = load_dotenv(find_dotenv())
print(os.environ["OPENAI_API_KEY"])

from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

from langchain.chains import LLMRequestsChain, LLMChain


llm = OpenAI(temperature=0, model_name='gpt-3.5-turbo')

d:\Anaconda3\envs\LLM\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\Anaconda3\envs\LLM\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\Anaconda3\envs\LLM\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


c:\Users\lenovo\Desktop\LangChainPlayGround\DeeperTutorials\.env
sk-lANo2jIeCWQt94UCCf5d16B7C32744279bF98b06C822D519


d:\Anaconda3\envs\LLM\lib\site-packages\langchain\llms\openai.py:201: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
d:\Anaconda3\envs\LLM\lib\site-packages\langchain\llms\openai.py:786: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [4]:
template = """Between >>> and <<< are the raw search result text from provided RSS URL.
Extract the answer to the question '{query}' or say "not found" if the information is not contained, and summarize all the information.
>>> {requests_result} <<<
Use the following JSON format to include all the titles:
{{
  "titles": [
    'aaa',
    'bbb',
  ]
}} 
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)


chain = LLMRequestsChain(llm_chain = LLMChain(llm=llm, prompt=PROMPT))

In [5]:
inputs = {
    "query": "What are all the titles in this RSS feed?",
    "url": "https://rss.nytimes.com/services/xml/rss/nyt/US.xml"
}


response = chain(inputs)
print(response['output'])


{
  "titles": [
    "After Maui’s Wildfire Horror, Residents Search for a Way Forward",
    "Trump Allies Likely to Be Booked at Notorious Atlanta Jail",
    "What We Know About the Wildfires in Hawaii",
    "With Racketeering Charges, Georgia Prosecutor Aims to ‘Tell the Whole Story’",
    "Texas Woman Charged With Threatening to Kill Judge in Trump Election Case",
    "North Carolina Bans Transgender Care for Minors as Republicans Override Veto",
    "Video Shows Man Shot by Philadelphia Officer Did Not Lunge as Police Said",
    "Tennessee Man Is Sentenced to Life Without Parole for 8 Murders",
    "Biden Celebrates Inflation Reduction Act’s Effects on Law’s First Anniversary",
    "All of Trump’s Lawyers and How Much They’re Paid in Legal Fees"
  ]
}


In [6]:
template = """在 >>> 和 <<< 之间是网页的返回的HTML内容。

网页是新浪财经A股上市公司的每季度股东信息表格。

请抽取参数请求的信息。每个截至日期作为JSON返回数据的date_of_quarter。因此，当表格中有多个截止日期时，返回数据应当包括所有的日期作为key。

>>> {requests_result} <<<
请使用如下的JSON格式返回数据
{{
  "date_of_quarter": [
    {{
      "holder_name": "a",
      "percentage": "50"
    }},
    {{
      "holder_name": "b",
      "percentage": "30"
    }},
  ]
}} 

例如，截至日期为2023-03-31，JSON数据应该是如下形式:

{{
  "2023-03-31": [
    {{
      "holder_name": "a",
      "percentage": "50"
    }},
    {{
      "holder_name": "b",
      "percentage": "30"
    }},
  ]
}}
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["requests_result"],
    template=template,
)

chain = LLMRequestsChain(llm_chain = LLMChain(llm=llm, prompt=PROMPT))

In [8]:
inputs = {
    "url": "https://vip.stock.finance.sina.com.cn/corp/go.php/vCI_StockHolder/stockid/600519/displaytype/30.phtml"
}

response = chain(inputs)
print(response['output'])

{
  "2023-06-30": [
    {
      "holder_name": "中国贵州茅台酒厂(集团)有限责任公司",
      "percentage": "54.07"
    },
    {
      "holder_name": "香港中央结算有限公司",
      "percentage": "7"
    },
    {
      "holder_name": "贵州省国有资本运营有限责任公司",
      "percentage": "4.54"
    },
    {
      "holder_name": "贵州茅台酒厂(集团)技术开发有限公司",
      "percentage": "2.22"
    },
    {
      "holder_name": "中央汇金资产管理有限责任公司",
      "percentage": "0.83"
    },
    {
      "holder_name": "深圳市金汇荣盛财富管理有限公司－金汇荣盛三号私募证券投资基金",
      "percentage": "0.71"
    },
    {
      "holder_name": "珠海市瑞丰汇邦资产管理有限公司－瑞丰汇邦三号私募证券投资基金",
      "percentage": "0.7"
    },
    {
      "holder_name": "中国证券金融股份有限公司",
      "percentage": "0.64"
    },
    {
      "holder_name": "中国工商银行－上证50交易型开放式指数证券投资基金",
      "percentage": "0.43"
    },
    {
      "holder_name": "中国银行股份有限公司－招商中证白酒指数分级证券投资基金",
      "percentage": "0.34"
    }
  ],
  "2023-03-31": [
    {
      "holder_name": "中国贵州茅台酒厂(集团)有限责任公司",
      "percentage": "54.06"
    },
    {
      "holder_name": "